<a href="https://colab.research.google.com/github/brahma1995/Flipkart_sentiment_analysis/blob/main/Copy_of_sentiment_analysis_flipkart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/nicknochnack/BERTSentiment.git

Cloning into 'BERTSentiment'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 1.61 KiB | 1.61 MiB/s, done.


In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 110.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.0 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [24]:
tokens = tokenizer.encode("After using Samsung Galaxy S23 Ultra for quite a bit as a daily driver and coming back to Samsung after a long time after using other smaller form factor phones like ONEPLUS 9, here's what I ",return_tensors='pt')

In [25]:
result = model(tokens)

In [26]:
result.logits

tensor([[-1.2469, -0.8955, -0.3896,  0.7303,  1.2422]],
       grad_fn=<AddmmBackward0>)

In [27]:
int(torch.argmax(result.logits))+1

5

In [28]:
headers = {
    "authority": "www.flipkart.com",
    "pragma": "no-cache",
    "cache-control": "no-cache",
    "dnt": "1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "sec-fetch-site": "none",
    "sec-fetch-mode": "navigate",
    "sec-fetch-dest": "document",
    "accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
}

In [29]:

cust_name = []
rev_content = []
for page in range(1,30):
  url = "https://www.flipkart.com/canon-eos-3000d-dslr-camera-1-body-18-55-mm-lens/product-reviews/itm6f665fea97bc2?pid=CAMF3DHJURPEMNRN&lid=LSTCAMF3DHJURPEMNRN8AVERV&marketplace=FLIPKART"+str(page)
  code = requests.get(url,headers=headers)
  if str(code) == "<Response [200]>":
    soup = BeautifulSoup(code.content,'html.parser')
    names = soup.find_all(class_='_2sc7ZR _2V5EHH')
    reviews = soup.find_all(class_="t-ZTKy")

    for i in range(len(names)):
        cust_name.append(names[i].get_text())
        
        rev_content.append(reviews[i].get_text().strip("READ MORE"))
reviews = rev_content

In [30]:
reviews

["product really great . I am totally sctified this product .really good camera for beggner .but flash button was missing that is not good .and capture is 18 mp wasn't good I was hopeed it will be 24 mp . it is also missing .n everything is good .",
 "Well Reviewing this after using for almost one month.I would prefer for the beginner photography. Simple without any complexity. It's best if you learn to use it at Manual mode",
 'Satisfied with the product.. Its a really good for all who wants to learn photography its a gem and love.... Go for itt....10/10',
 'The camera simply awesome..and it is a best budget DSLR and begginer DSLR for first time user...auto focus mode works good in daylight',
 "Good oneBest for beginner photographsNot recommend for videography because it is not good for videography or vloging porpes it is heavy and bulky.So if you're priority is photography it's a nice and value for money product.",
 "The product is very good But if I take another lens of 55-250mm len

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])


In [ ]:
df['review'].iloc[0]


"product really great . I am totally sctified this product .really good camera for beggner .but flash button was missing that is not good .and capture is 18 mp wasn't good I was hopeed it will be 24 mp . it is also missing .n everything is good .READ MORE"

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt',truncation=True)
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['review'].iloc[1])

4

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))
df['cust_name'] = cust_name

In [ ]:
df

,review,sentiment,cust_name
0,product really great . I am totally sctified t...,3,Flipkart Customer
1,Well Reviewing this after using for almost one...,4,Kaushik Das
2,Satisfied with the product.. Its a really good...,5,MUSSARAT SHAMIM
3,The camera simply awesome..and it is a best bu...,5,Pavithran V
4,Good oneBest for beginner photographsNot recom...,4,Abik Peravan
...,...,...,...
285,The product is very good But if I take another...,4,Flipkart Customer
286,Camera is good for beginner. But not getting...,3,Tejpal Singh Bhandari
287,Excellent for someone who is new into photogra...,4,Khirod Sahoo
288,Verry Nice and for My Opinion is great camera ...,5,Amol Kharate


In [ ]:
df['review'].iloc[3]

"It was ok. Had coffee with my friends. I'm new in the area, still need to discover new places."